In [ ]:
import re
import tkinter as tk
from tkinter.scrolledtext import ScrolledText
from tkinter import Menu
from spellchecker import SpellChecker

class SpellingChecker:
    def __init__(self):
        self.spell = SpellChecker()
        self.root = tk.Tk()
        self.root.geometry("600x500")

        self.text = ScrolledText(self.root, font=("Arial", 14))
        self.text.bind("<KeyRelease>", self.check)
        self.text.pack()
        self.text.bind("<Button-3>", self.show_suggestions)

        self.old_spaces = 0

        self.root.mainloop()

    def check(self, event):
        content = self.text.get("1.0", tk.END)
        space_count = content.count(" ")

        for tag in self.text.tag_names():
            self.text.tag_delete(tag)

        if space_count != self.old_spaces:
            self.old_spaces = space_count
            for word in content.split(" "):
                cleaned_word = re.sub(r"[^\w]", "", word.lower())
                if cleaned_word and cleaned_word not in self.spell:
                    start_pos = f"1.{content.find(word)}"
                    end_pos = f"1.{content.find(word) + len(word)}"
                    self.text.tag_add(word, start_pos, end_pos)
                    self.text.tag_config(word, foreground="red")

    def show_suggestions(self, event):
        self.context_menu = Menu(self.root, tearoff=0)
        try:
            word_position = self.text.index(f"@{event.x},{event.y}")
            word = self.text.get(word_position + " wordstart", word_position + " wordend")
            suggestions = self.spell.candidates(word)
            for suggestion in suggestions:
                self.context_menu.add_command(label=suggestion, command=lambda s=suggestion: self.replace_word(word_position, s))
            self.context_menu.add_separator()
            self.context_menu.add_command(label="Cancel", command=self.context_menu.unpost)
            self.context_menu.post(event.x_root, event.y_root)
        except Exception as e:
            print(e)

    def replace_word(self, word_position, suggestion):
        word_start = word_position + " wordstart"
        word_end = word_position + " wordend"
        self.text.delete(word_start, word_end)
        self.text.insert(word_start, suggestion)
        self.context_menu.unpost()

SpellingChecker()
